In [1]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
from scipy.stats.stats import kendalltau
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras.utils.np_utils import to_categorical
from sklearn.utils import class_weight
from sklearn.metrics import log_loss
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D, Conv1DTranspose
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LSTM, Masking
from tensorflow.keras.layers import MaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras import activations
sns.set()
from sklearn.model_selection import KFold

In [2]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import pandas as pd
from scipy import *

data = np.empty(shape=[0, 222])

all_data = glob.glob('./drive/MyDrive/compsci/all_data.csv')

for j in all_data:
    print('Loading ', j)
    csvrows = np.loadtxt(j, delimiter=',')
    data = np.append(data, csvrows, axis=0)

print(data.shape)

Loading  ./drive/MyDrive/compsci/all_data.csv
(151452, 222)


In [3]:
X = data[:,:-2]
y = data[:,-2]

inputs = X.reshape(X.shape + (1,1))
targets = to_categorical(y)

print(inputs.shape)
print(targets.shape)

(151452, 220, 1, 1)
(151452, 9)


In [4]:
def showResults(test, pred):
    #target_names = ['positive', 'negative']
    # print(classification_report(test, pred, target_names=target_names))
    accuracy = accuracy_score(test, pred)
    precision=precision_score(test, pred, average='weighted')
    f1Score=f1_score(test, pred, average='weighted') 
    #loss=log_loss(test,pred)
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1Score))

In [5]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=6, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1

verbose, epoch, batch_size = 1, 1, 256
for train, test in kfold.split(inputs, targets):

  lstmmodel = Sequential()
  lstmmodel.add(LSTM(128, return_sequences=True, input_shape=(inputs[train].shape[1],1)))
  lstmmodel.add(LSTM(9, return_sequences=True))
  lstmmodel.add(MaxPooling1D(pool_size=2))
  lstmmodel.add(Flatten())
  lstmmodel.add(Dense(512, activation='relu'))
  lstmmodel.add(Dense(128, activation='relu'))    
  lstmmodel.add(Dense(32, activation='relu'))
  lstmmodel.add(Dense(9, activation='softmax'))
  lstmmodel.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
  
  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  modelhistory= lstmmodel.fit(inputs[:,:,:,0][train], targets[train], epochs=epoch, 
                             verbose=verbose, validation_split=0.2, 
                             batch_size = batch_size)
  print(fold_no)
  lstmpredictions = lstmmodel.predict(inputs[:,:,:,0][test], verbose=1) 
  lstm_predict=np.argmax(lstmpredictions,axis=1)
  lstm_actual_value=np.argmax(targets[test],axis=1)
  LSTM_cf_m = showResults(lstm_actual_value, lstm_predict)

  # Increase fold number
  fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
395/395 [==============================] - 21s 42ms/step - loss: 0.3818 - accuracy: 0.9034 - val_loss: 0.5627 - val_accuracy: 0.8516
1
789/789 [==============================] - 9s 10ms/step
Accuracy  : 0.9548371761350131
Precision : 0.9542200350061847
f1Score : 0.951247262503336
------------------------------------------------------------------------
Training for fold 2 ...
395/395 [==============================] - 19s 41ms/step - loss: 0.3986 - accuracy: 0.9037 - val_loss: 0.6624 - val_accuracy: 0.8135
2
789/789 [==============================] - 9s 10ms/step
Accuracy  : 0.9451707471674193
Precision : 0.9449687270887588
f1Score : 0.9413274787169472
------------------------------------------------------------------------
Training for fold 3 ...
395/395 [==============================] - 20s 41ms/step - loss: 0.3839 - accuracy: 0.9013 - val_loss: 0.5159 - val_accuracy: 0.8452
3
789/789 [==

In [6]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=6, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1

verbose, epoch, batch_size = 1, 1, 64
for train, test in kfold.split(inputs, targets):

  # Define the model architecture
  cnnmodel = Sequential()
  cnnmodel.add(Conv1D(filters=128, kernel_size=16,padding='same', activation='relu',input_shape=(inputs[train].shape[1],1)))
  cnnmodel.add(BatchNormalization())
  cnnmodel.add(Conv1D(filters=32, kernel_size=16,padding='same', activation='relu'))
  cnnmodel.add(BatchNormalization())
  cnnmodel.add(Conv1D(filters=9, kernel_size=16,padding='same', activation='relu'))
  cnnmodel.add(MaxPooling1D(pool_size=2,padding='same'))
  cnnmodel.add(Flatten())
  cnnmodel.add(Dense(512, activation='relu'))
  cnnmodel.add(Dense(128, activation='relu'))
  cnnmodel.add(Dense(32, activation='relu'))
  cnnmodel.add(Dense(9, activation='softmax'))
  cnnmodel.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])


  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  modelhistory= cnnmodel.fit(inputs[:,:,:,0][train], targets[train], epochs=epoch, 
                             verbose=verbose, validation_split=0.2, 
                             batch_size = batch_size)
  print(fold_no)
  cnnpredictions = cnnmodel.predict(inputs[:,:,:,0][test], verbose=1) 
  cnn_predict=np.argmax(cnnpredictions,axis=1)
  cnn_actual_value=np.argmax(targets[test],axis=1)
  CNN_cf_m = showResults(cnn_actual_value, cnn_predict)

  # Increase fold number
  fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
1578/1578 [==============================] - 14s 8ms/step - loss: 0.1701 - accuracy: 0.9564 - val_loss: 0.5807 - val_accuracy: 0.8501
1
789/789 [==============================] - 2s 2ms/step
Accuracy  : 0.9555502733539339
Precision : 0.9531197666111557
f1Score : 0.950002298046538
------------------------------------------------------------------------
Training for fold 2 ...
1578/1578 [==============================] - 13s 8ms/step - loss: 0.1870 - accuracy: 0.9506 - val_loss: 0.6543 - val_accuracy: 0.8320
2
789/789 [==============================] - 2s 2ms/step
Accuracy  : 0.9544806275255526
Precision : 0.9530333031294481
f1Score : 0.9522118546963653
------------------------------------------------------------------------
Training for fold 3 ...
1578/1578 [==============================] - 13s 8ms/step - loss: 0.1923 - accuracy: 0.9494 - val_loss: 0.5041 - val_accuracy: 0.8649
3
789/789 [=